# <center>Other FBAs and Deletions</center>#

### pFBA ###

We've seen that FBA generates A steady state flux distribution where biomass is maximized. FVA has taught us that there are multiple possible solutions to an FBA problem. You may observe in FVA results that some reactions's fluxes can span the entire range of possible flux values. This is sometimes, a result of thermodynamically infeasible loops in metabolic networks. Several possible solutions have been proposed to eliminate such loops. One such method is pFBA - parsimonious Flux balance analysis - minimization of the sum of absolute value of fluxes subject to the normal FBA constraints. Let's see how pFBA works

In [ ]:
import cameo
import cobra
from cameo.strain_design import OptKnock
from cameo.flux_analysis import lmoma,room

In [ ]:
model = cameo.models.bigg.e_coli_core
fbaresult = cameo.fba(model)
pfbaresult = cameo.pfba(model)
print("Sum of FBA Fluxes",abs(fbaresult.data_frame.flux).sum())
print("Sum of pFBA Fluxes",abs(pfbaresult.data_frame.flux).sum())


The pFBA problem here, is not that useful since the sum of fluxes is reduced only slightly. This means that FBA on the model already predicted a flux distribution without any loops. However, there are some metabolic models that will result in "loopy" fluxes upon running FBA. pFBA is useful in these cases to get realistic flux distributions. In fact, it may be useful to always run pFBA/loopless FBA in addition to FBA to ensure we don't get any infeasible flux distributions.

### Simulating Gene/Reaction Knockouts ###

We may wish to model metabolism in a mutant strain where a particular gene has been knocked out. Cameo/cobra allow us to delete reactions by setting their upper and lower bounds to zero. To knockout a gene, use the syntax: <i>model.genes."GeneID".knock_out()</i>. To knockout a reaction, use the syntax: <i>model.reactions."ReactionID".knock_out()</i>

In [ ]:
model = cameo.models.bigg.e_coli_core
print('Growth Rate for WT:', cameo.fba(model).fluxes['BIOMASS_Ecoli_core_w_GAM'])

#The with command in python allows you to reversibly modify variables. After the block within the with command,
# the variable returns to its original form
with model:
    model.reactions.PFK.knock_out()
    print('Growth Rate with pfk knocked out:', cameo.fba(model).fluxes['BIOMASS_Ecoli_core_w_GAM'])

In the above example, we predict the growth rate of wild type <i>E.coli</i> and compare it to the growth rate of <i>E.coli</i> with pfk (phospho-fructokinase) knocked out. 

Here, FBA on knockout strains can be used reliably only to predict whether a particular gene/reaction is essential to the model. Hence, in this case, FBA predicts that <i>E.coli</i> can grow even if pfk is knocked out.

Cobra helps us predict which reactions are essential for the model. For this, cobra deletes each reaction in the model and solves the model using FBA to predict the growth rate. If the growth rate is zero or near zero, we can conclude that the particular reaction is essential. If the solver status is "infeasible", cobra was unable to perform FBA on the model. Usually, the solver status will be infeasible for cases where glucose transport reactions are blocked.

In [ ]:
deletion_result = cobra.flux_analysis.single_reaction_deletion(model)


The above line of code displays the objective reaction flux and solver status for each knocked out reaction. If we're interested in how many essential reactions there are, we simply look for those knockouts where the objective reaction flux is close to zero.

In [ ]:
lethal_deletions = deletion_result[deletion_result['growth']<1e-5]
print("No. of essential reactions in the core model: ",len(lethal_deletions)-1)


Note that the number of essential reactions changes with environmental conditions and flux bounds on other reactions. This means that the number of essential reactions under aerobic conditions will be different from the number of essential reactions under anaerobic conditions

### Predicting Gene Knockouts - OptKnock###

Why would we knock out genes in the first place? It helps us modify metabolism in an organism. We can make a microbe produce a certain chemical that it normally does not or increase the yield of a certain chemical at its optimum growth conditions. OptKnock is one such method that predicts knockout strategies that allows us to couple production of a certain chemical to an organism's growth. Let's see how it works for the case of ethanol production in E.coli. 

Note: DO NOT ATTEMPT TO RUN OPTKNOCK ON GENOME SCALE MODELS on your laptop. It will take forever to run using the free solvers and low CPU power. If you are interested in predicting knockout strategies for genome scale models, you may need to gain access to very powerful computers.


In [ ]:
optknock = cameo.strain_design.OptKnock(model,fraction_of_optimum=0.1)
result = optknock.run(max_knockouts = 3, target="EX_lac__D_e", biomass="BIOMASS_Ecoli_core_w_GAM")
result.plot()
display(result.data_frame)

In this case, OptKnock predicts 3 possible single gene deletions that couple growth to production of ethanol in E.coli's core metabolism. Hence, IF these deletions are made and E.coli grows at its optimum growth rate, lactic acid will be produced. Guess what reactions have been predicted by OptKnock - use BiGG to figure out which metabolic processes they're involved in.

Now, let's take a core model of E.coli and confirm that this actually happens:

In [ ]:
model = cameo.models.bigg.e_coli_core
model.reactions.EX_co2_e.lower_bound = 0
result = cameo.phenotypic_phase_plane(model,variables=[model.reactions.BIOMASS_Ecoli_core_w_GAM],
                                      objective=model.reactions.EX_etoh_e,points=10)
print("WildType")
result.plot()
with model:
    model.reactions.CYTBD.knock_out()
    result = cameo.phenotypic_phase_plane(model,variables=[model.reactions.BIOMASS_Ecoli_core_w_GAM],
                                      objective=model.reactions.EX_etoh_e,points=10)
    print("CYTBDMutant")
    result.plot()

Running FBA on a mutant will optimize for the growth rate and therefore result in a flux through the growth coupled reaction. However, this may not be observed experimentally. (WHY? Cells are optimize for growth and at maximum growth, there should be flux through the target reaction. However, flux through the target reaction is sometimes lower than what is expected. Why does this not happen?

### Simulating Gene Deletions - MOMA/ Minimization of Metabolic Adjustment ###

MOMA seems to predict the phenotype of deletion mutants better than pFBA/FBA or other growth optimization strategies.

In [ ]:
with model:
    ref_flux = cameo.fba(model).fluxes
    model.reactions.EX_o2_e.knock_out()
    moma_result = cameo.flux_analysis.simulation.lmoma(model,reference = ref_flux)
    fba_result = cameo.fba(model)
    fba_biomass = fba_result.fluxes['BIOMASS_Ecoli_core_w_GAM']
    fba_etoh = fba_result.fluxes['EX_etoh_e']
    moma_biomass = moma_result.fluxes['BIOMASS_Ecoli_core_w_GAM']
    moma_etoh = moma_result.fluxes['EX_etoh_e']
    result = cameo.phenotypic_phase_plane
    result = cameo.phenotypic_phase_plane(model,variables=[model.reactions.BIOMASS_Ecoli_core_w_GAM],
                                      objective=model.reactions.EX_etoh_e,points=10)
    result.plot(points=[(fba_biomass,fba_etoh), (moma_biomass,moma_etoh)], points_colors=["green", "red"])
    print("Growth rate predicted by FBA for an anaerobic strain: ", fba_biomass)
    print("Ethanol flux predicted by FBA for an anaerobic strain: ", fba_etoh)
    print("Growth rate predicted by MOMA for an anaerobic strain: ", moma_biomass)
    print("Ethanol flux predicted by MOMA for an anaerobic strainP ", moma_etoh)
    
    


In [ ]:
model = cameo.models.bigg.e_coli_core
with model:
    for reaction in model.reactions:
        if reaction.upper_bound == 1000:
            reaction.upper_bound = 99999999
        if reaction.lower_bound == -1000:
            reaction.lower_bound = -99999999
    ref_flux = cameo.fba(model).fluxes
    model.reactions.EX_o2_e.knock_out()
    room_result = cameo.flux_analysis.simulation.room(model,reference = ref_flux)
    fba_result = cameo.fba(model)
    fba_biomass = fba_result.fluxes['BIOMASS_Ecoli_core_w_GAM']
    fba_etoh = fba_result.fluxes['EX_etoh_e']
    room_biomass = room_result.fluxes['BIOMASS_Ecoli_core_w_GAM']
    room_etoh = room_result.fluxes['EX_etoh_e']
    result = cameo.phenotypic_phase_plane
    result = cameo.phenotypic_phase_plane(model,variables=[model.reactions.BIOMASS_Ecoli_core_w_GAM],
                                      objective=model.reactions.EX_etoh_e,points=10)
    result.plot(points=[(fba_biomass,fba_etoh), (room_biomass,room_etoh)], points_colors=["green", "red"])
    print("Growth rate predicted by FBA for an anaerobic strain: ", fba_biomass)
    print("Ethanol flux predicted by FBA for an anaerobic strain: ", fba_etoh)
    print("Growth rate predicted by ROOM for an anaerobic strain: ", room_biomass)
    print("Ethanol flux predicted by ROOM for an anaerobic strainP ", room_etoh)
    
    
